## Quality Control Script for QGIS Points Layer
by Tara Wu, Spring 2025

<div class="alert alert-block alert-info">

<b>Purpose:</b>  This script performs a series of quality checks for point features in QGIS. Specifically, it checks for the following: 
<ul>
    <li>nulls in specific fields </li>
    <li>invalid dates</li>
    <li>orphaned related records</li>
    <li>missing related records </li>
    <li> duplicates due to sync error,</li> 
    <li>inputs not in domains, repetitive attributes per user </li>
    <li>offline data has been synced</li>
    <li>proximity to trails/features</li> 
    <li>matching collector and region</li>
</ul>
</div>

In [ ]:
# === PARAMETERS ===

# === import modules ===
import datetime
import importlib
import logging
import os
import sys

import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union

# check for required packages
required_packages = [
    "os",
    "datetime",
    "logging",
    "sys",
    "pandas",
    "geopandas",
    "shapely",
]

for pkg in required_packages:
    if importlib.util.find_spec(pkg) is None:
        print(f"Missing package '{pkg}'. Install via: pip install {pkg}")
        sys.exit(1)

# === output setup ===
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
DATA_FOLDER = os.path.join(BASE_DIR, "data")
OUTPUT_FOLDER = os.path.join(BASE_DIR, "outputs")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# input files
POINTS_FILE = os.path.join(DATA_FOLDER, "points.gpkg")  # Update with your layer
RELATED_TABLE_FILE = os.path.join(DATA_FOLDER, "related.csv")  # Optional related table

# project start date
PROJECT_START_YEAR = 2025
PROJECT_START_MONTH = 5
PROJECT_START_DAY = 5

# thresholds / buffers
BUFFER_FT = 100
THRESHOLD = 0.9  # for repetitive values

# required fields
POINTS_REQUIRED_FIELDS = ["GlobalID", "created_user", "created_date", "SHAPE"]
RELATED_REQUIRED_FIELDS = ["defGlobalID", "Feature", "Feature_Action"]

# fields for sync/repetitive checks
POINTS_SYNC_FIELDS = ["created_date", "created_user"]
RELATED_SYNC_FIELDS = ["CreationDate", "Creator"]

POINTS_REP_FIELDS = ["Evaluation_Code", "Deficiency_Length"]
RELATED_REP_FIELDS = ["Feature"]

# domains
POINTS_DOMAINS = {
    "State": ["MA", "ME", "CT", "NH", "VT"],
    "Club": ["AMC", "AMC-CT", "AMC-WMA", "DOC", "GMC", "MATC", "RMC"],
    "Evaluation_Code": ["Low", "Moderate", "High", None],
    "OnsiteMaterials": ["Yes", "No", "Maybe", None],
    "ConsiderRelocation": ["Yes", "No", None],
}

RELATED_DOMAINS = {
    "Feature": ["Trail", "Bridge", "Shelter", None],
    "FeatureAction": ["Build Add", "Repair Replace", "Remove", None],
    "Units": ["Each", "LinearFeet", "SquareFeet", None],
    "onsitematerials": ["Yes", "No", "Maybe", None],
}

# collector-region dictionary
COLLECTOR_DICT = {
    "twu_ATConservancy": ["MATC"],
    "userB": ["RMC", "AMC", "DOC"],
    "userC": ["GMC"],
    "userD": ["AMC-WMA"],
    "userE": ["AMC-CT"],
}

# output column order
POINTS_ERROR_ORDER = [
    "error_type",
    "error_desc",
    "OBJECTID",
    "GlobalID",
    "created_user",
    "created_date",
    "Deficiency_Length",
    "Evaluation_Code",
    "ConsiderRelocation",
    "Notes",
    "OnsiteMaterials",
    "RelativeLinearLocation",
    "SHAPE",
    "MileMarker",
    "State",
    "LandOwner",
    "OwnershipType",
    "Club",
]

RELATED_ERROR_ORDER = [
    "error_type",
    "error_desc",
    "OBJECTID",
    "GlobalID",
    "defGlobalID",
    "CreationDate",
    "Creator",
    "Feature",
    "Feature_Action",
    "Quantity",
    "Units",
    "onsitematerials",
]

WRITE_LOG = True

# === LOGGING SETUP ===
FILE_PATH = os.path.join(
    OUTPUT_FOLDER, f"{datetime.datetime.today().strftime('%Y-%m-%d')}_QC_summary.txt"
)
handlers = [logging.StreamHandler(sys.stdout)]
if WRITE_LOG:
    handlers.append(logging.FileHandler(FILE_PATH, mode="w", encoding="utf-8"))

logging.basicConfig(level=logging.INFO, format="%(message)s", handlers=handlers)

# === HELPER FUNCTIONS ===
def check_nulls(df, df_name, fields, error_list, summary_rows):
    any_nulls = False
    for field in fields:
        if field in df.columns:
            nulls = df[df[field].isnull()].copy()
            if not nulls.empty:
                any_nulls = True
                nulls.loc[:, "error_type"] = "nulls"
                nulls.loc[:, "error_desc"] = f"NULL in {df_name} field: {field}"
                error_list.append(nulls)
                msg = f"{df_name}.{field}: {len(nulls)} nulls"
                logging.info("        " + msg)
                summary_rows.append(msg)
        else:
            logging.info(f"        {field} not in {df_name}")
    if not any_nulls:
        msg = f"0 null errors found in {df_name}"
        logging.info("        " + msg)
        summary_rows.append(msg)


def check_dates(df, df_name, date_field, error_list, summary_rows):
    project_start_date = datetime.datetime(
        PROJECT_START_YEAR, PROJECT_START_MONTH, PROJECT_START_DAY
    )
    today = datetime.datetime.today()
    df[date_field] = pd.to_datetime(df[date_field], errors="coerce")
    invalid_dates = df[
        (df[date_field] > today) | (df[date_field] < project_start_date)
    ].copy()
    if not invalid_dates.empty:
        invalid_dates.loc[:, "error_type"] = "dates"
        invalid_dates.loc[:, "error_desc"] = "Invalid observation date"
        error_list.append(invalid_dates)
    msg = f"{len(invalid_dates)} records with invalid dates in {df_name}"
    logging.info("        " + msg)
    summary_rows.append(msg)


def check_domains(df, df_name, domain_dict, error_list, summary_rows):
    any_domain_errors = False
    for d in domain_dict:
        if d in df.columns:
            valid_values = domain_dict[d]
            invalid = df[~(df[d].isin(valid_values) | df[d].isnull())].copy()
            if not invalid.empty:
                any_domain_errors = True
                invalid.loc[:, "error_type"] = "domains"
                invalid.loc[:, "error_desc"] = invalid[d].apply(
                    lambda x: f"'{x}' not in domains for {d}"
                )
                error_list.append(invalid)
                msg = f"Invalid entries for {d} in {df_name}: {list(invalid[d].dropna().unique())}"
                logging.info("        " + msg)
                summary_rows.append(msg)
    if not any_domain_errors:
        msg = f"0 domain errors found in {df_name}"
        logging.info("        " + msg)
        summary_rows.append(msg)


def check_repetitive_values(
    df, df_name, fields, user_field, threshold, error_list, summary_rows
):
    grouped = df.groupby(user_field)
    rep_error_count = 0
    for user, group in grouped:
        for field in fields:
            if field in group.columns:
                value_counts = group[field].value_counts(normalize=True)
                dominant_values = value_counts[value_counts > threshold]
                for val in dominant_values.index:
                    rep_errors = group[group[field] == val].copy()
                    rep_errors.loc[:, "error_type"] = "repetitive"
                    rep_errors.loc[:, "error_desc"] = (
                        f"{user} repeated '{val}' in {df_name}.{field} over {int(threshold*100)}% of time"
                    )
                    error_list.append(rep_errors)
                    rep_error_count += len(rep_errors)
    msg = (
        f"{rep_error_count} potentially repetitive inputs in {df_name}"
        if rep_error_count
        else f"0 potentially repetitive inputs in {df_name}"
    )
    logging.info("        " + msg)
    summary_rows.append(msg)


# --- Future helper functions: proximity checks, collector-region checks, etc. ---
def check_proximity(df, other_layers, buffer_distance, error_list, summary_rows):
    """
    Placeholder for proximity checks to trails or features.
    `other_layers` can be a list of GeoDataFrames.
    """
    pass


def check_collector_region(df, collector_dict, error_list, summary_rows):
    """
    Placeholder for collector-region matching QC.
    """
    pass


# === SUMMARY GENERATION ===
start_time = datetime.datetime.now()
summary_rows = []
points_errors = []
related_errors = []

# load local data
points_gdf = gpd.read_file(POINTS_FILE)
related_df = (
    pd.read_csv(RELATED_TABLE_FILE)
    if os.path.exists(RELATED_TABLE_FILE)
    else pd.DataFrame()
)

# summarize quality checks
summary_rows.append("Null errors:")
check_nulls(points_gdf, "points", POINTS_REQUIRED_FIELDS, points_errors, summary_rows)
if not related_df.empty:
    check_nulls(
        related_df, "related", RELATED_REQUIRED_FIELDS, related_errors, summary_rows
    )

summary_rows.append("Date errors:")
check_dates(points_gdf, "points", "created_date", points_errors, summary_rows)

summary_rows.append("Domain errors:")
check_domains(points_gdf, "points", POINTS_DOMAINS, points_errors, summary_rows)
if not related_df.empty:
    check_domains(related_df, "related", RELATED_DOMAINS, related_errors, summary_rows)

summary_rows.append("Repetitive value errors:")
check_repetitive_values(
    points_gdf,
    "points",
    POINTS_REP_FIELDS,
    "created_user",
    THRESHOLD,
    points_errors,
    summary_rows,
)
if not related_df.empty:
    check_repetitive_values(
        related_df,
        "related",
        RELATED_REP_FIELDS,
        "Creator",
        THRESHOLD,
        related_errors,
        summary_rows,
    )

# output results
output_xlsx = os.path.join(
    OUTPUT_FOLDER, f"{datetime.datetime.today().strftime('%Y-%m-%d')}_QC_report.xlsx"
)
with pd.ExcelWriter(output_xlsx) as writer:
    (
        pd.concat(points_errors).to_excel(writer, sheet_name="points", index=False)
        if points_errors
        else pd.DataFrame().to_excel(writer, sheet_name="points")
    )
    (
        pd.concat(related_errors).to_excel(writer, sheet_name="related", index=False)
        if related_errors
        else pd.DataFrame().to_excel(writer, sheet_name="related")
    )
    pd.DataFrame(summary_rows, columns=["summary"]).to_excel(
        writer, sheet_name="summary", index=False
    )

end_time = datetime.datetime.now()
logging.info(
    f"\nQC completed in {end_time - start_time}. Excel report saved to {output_xlsx}"
)
if WRITE_LOG:
    logging.info(f"Log file saved to {FILE_PATH}")
